In [1]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns


In [7]:
merged = pd.read_csv('final.csv' , index_col = 'application_date')
merged.head()

,case_count,day,segment,y,m,d
application_date,,,,,,
2017-04-01,299.0,5,1,2017,4,1
2017-04-01,897.0,5,2,2017,4,1
2017-04-02,605.0,6,2,2017,4,2
2017-04-03,42.0,0,1,2017,4,3
2017-04-03,2016.0,0,2,2017,4,3


In [8]:
# seg = merged[merged['segment'] == 1]

In [9]:
# %matplotlib notebook

In [10]:
#plt.plot(seg.index , seg.case_count)

In [11]:
def mean_encodings(df , cols) :
    for col in cols :
        df['{}_enc'.format(col)] = df.groupby(col)['case_count'].transform('mean')
        #df['{}_enc'.format(col)].fillna(0.4453, inplace=True)
    return df

cols = [ 'day' , 'segment' , 'd' , 'm' , 'y']


merged = mean_encodings(merged , cols)
# merged.index.value_counts()

In [12]:
df = merged[merged.index  < '2019-04-01'].dropna()



val = merged[(merged.index  >= '2019-04-01') & (merged.index  >= '2019-07-05')].dropna()


test = merged[(merged.index  >= '2019-07-05')]

In [13]:
X_train = df.drop(['case_count'] , axis = 1)
y_train = df['case_count'].astype(int)

X_val = val.drop(['case_count'] , axis = 1)
y_val = val['case_count'].astype(int)


In [14]:
categorical = ['m' , 'd' , 'day' ,'y' , 'segment']

def column_index(df, query_cols):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols, query_cols, sorter=sidx)]
categorical_features_indices = column_index(X_train, categorical)

In [13]:
#train.describe()
from catboost import CatBoostRegressor

In [14]:
model =  CatBoostRegressor(iterations=3000,
                             learning_rate=0.001,
                             depth=16,
                             eval_metric='MAPE',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50,
                             task_type = "GPU",
                             devices='0:1',
                             
                             od_wait=100)

In [15]:
model.fit(X_train, y_train,
                 #eval_set=(X_val, y_val),
                  cat_features=categorical_features_indices,
                 use_best_model=True)

You should provide test set for use best model. use_best_model parameter has been switched to false value.


0:	learn: 26.3274124	total: 70.1ms	remaining: 3m 30s
50:	learn: 25.4101325	total: 864ms	remaining: 50s
100:	learn: 24.5310348	total: 2.92s	remaining: 1m 23s
150:	learn: 23.7039565	total: 5.4s	remaining: 1m 41s
200:	learn: 22.9238700	total: 6.32s	remaining: 1m 27s
250:	learn: 22.1732803	total: 8.71s	remaining: 1m 35s
300:	learn: 21.4651405	total: 11.2s	remaining: 1m 40s
350:	learn: 20.7814317	total: 13.1s	remaining: 1m 38s
400:	learn: 20.1290309	total: 14.2s	remaining: 1m 31s
450:	learn: 19.5133382	total: 17.5s	remaining: 1m 38s
500:	learn: 18.9209833	total: 21s	remaining: 1m 44s
550:	learn: 18.3672677	total: 25.4s	remaining: 1m 52s
600:	learn: 17.8252242	total: 26.9s	remaining: 1m 47s
650:	learn: 17.3110549	total: 33.2s	remaining: 1m 59s
700:	learn: 16.8198494	total: 37.9s	remaining: 2m 4s
750:	learn: 16.3460847	total: 41s	remaining: 2m 2s
800:	learn: 15.8882477	total: 46.2s	remaining: 2m 6s
850:	learn: 15.4644543	total: 55.6s	remaining: 2m 20s
900:	learn: 15.0452592	total: 1m 2s	remai

In [15]:
realt = pd.read_csv('./test.csv')# , index_col = 'application_date')
realt.head()

,Unnamed: 0,application_date,segment,day
0,0,2019-07-06,1,5
1,1,2019-07-07,1,6
2,2,2019-07-08,1,0
3,3,2019-07-09,1,1
4,4,2019-07-10,1,2


In [16]:
mer1 = merged.reset_index()

In [17]:


ans = pd.merge(realt , mer1 ,  how = 'left' , on = ['application_date' , 'segment' , 'day']).drop(['case_count' , 'Unnamed: 0'] , axis = 1)
ans = ans.set_index(['application_date']).sort_index()
ans.head()

,segment,day,y,m,d,day_enc,segment_enc,d_enc,m_enc,y_enc
application_date,,,,,,,,,,
2019-07-06,1,5,2019,7,6,9086.821277,2714.689826,4443.250000,9785.493243,10438.507692
2019-07-07,1,6,2019,7,7,6192.443966,2714.689826,4411.296296,9785.493243,10438.507692
2019-07-08,1,0,2019,7,8,9170.556962,2714.689826,4632.981132,9785.493243,10438.507692
2019-07-09,1,1,2019,7,9,9624.451477,2714.689826,5145.169811,9785.493243,10438.507692
2019-07-10,1,2,2019,7,10,9806.423729,2714.689826,5590.740741,9785.493243,10438.507692


In [18]:
# na_free = merged.dropna()
# only_na = merged[~merged.index.isin(na_free.index)]

In [19]:
# test = only_na.sort_index()
# test = test.drop(['case_count'] ,axis = 1)
# test.head()

In [21]:
y_test = model.predict(ans)
y_test

array([ 3459.11261783,  3308.39401312,  3640.02375258,  3680.31009201,
        3789.25062499,  4038.65631868,  3999.11417526,  3945.91046265,
        3373.02159991,  4038.69685801,  4034.98500384,  4068.62762317,
        4048.33534217,  4018.57810532,  3958.10257768,  3353.22093103,
        4036.17439608,  4014.3640097 ,  4066.81956548, 23680.89567354,
        4037.9173591 , 22246.8598627 , 20395.41661902,  3974.10332615,
        3899.75413447, 15786.00964109,  9859.33586812,  3487.31548223,
        7531.93425   ,  3682.67847123,  6901.69583686,  3537.54515914,
        6263.29469369,  6206.40880169,  7441.9512427 ,  3760.38864135,
        3724.58687909,  7384.27839347,  3713.35792328,  7284.33815247,
        6402.58397374,  3575.99705272,  3838.16976008,  7339.35718445,
        3855.15392687,  7554.81188157,  7530.82395352,  3851.21789731,
        3889.83795723,  7681.36768934,  8098.81247074,  3929.45855486,
        3897.68394246,  8292.70791126,  3565.91226518, 14363.84040679,
      

In [22]:
realt['case_count'] = y_test

In [23]:
realt = realt.rename(columns={"Unnamed: 0": "id"}).drop('day' , axis = 1)


In [24]:
realt['id'] = realt['id'].astype(int) + 1

In [25]:
realt = realt.set_index(['id'])
realt.head()

,application_date,segment,case_count
id,,,
1,2019-07-06,1,3459.112618
2,2019-07-07,1,3308.394013
3,2019-07-08,1,3640.023753
4,2019-07-09,1,3680.310092
5,2019-07-10,1,3789.250625


In [26]:
realt.to_csv('solution.csv')

In [20]:
train_y = y_train
test_y = y_val

In [21]:
train_X = X_train
train_X = train_X.values.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test 
test_X = test_X.values.reshape((test_X.shape[0], 1, test_X.shape[1]))

In [22]:
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(1440, 1, 10) (1440,) (200, 1, 11) (20,)


In [33]:
print(tf.__version__)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\nitesh chaudhry\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-33-65f07e81e169>", line 1, in <module>
    print(tf.__version__)
AttributeError: module 'tensorflow' has no attribute '__version__'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\nitesh chaudhry\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'AttributeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\nitesh chaudhry\appdata\local\programs\python\python36\lib\site-packages\tensorflow_core\python\pywrap_tensorflow

AttributeError: module 'tensorflow' has no attribute '__version__'

In [32]:
import tensorflow as tf


In [27]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=60, kernel_size=5,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=[None, 1]),
  tf.keras.layers.LSTM(60, return_sequences=True),
  tf.keras.layers.LSTM(60, return_sequences=True),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 400)
])


optimizer = tf.keras.optimizers.SGD(lr=1e-5, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mape"])
history = model.fit(X_train,epochs=150)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\nitesh chaudhry\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-ffb1f0c31954>", line 1, in <module>
    model = tf.keras.models.Sequential([
AttributeError: module 'tensorflow' has no attribute 'keras'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\nitesh chaudhry\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'AttributeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\nitesh chaudhry\appdata\local\programs\python\python36\lib\site-packages\tensorflow_core\python\pywrap_t

AttributeError: module 'tensorflow' has no attribute 'keras'